In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [13]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [25]:
from threading import Thread

class StreamingThread(Thread):
    def __init__(self, ssc):
        Thread.__init__(self)
        self.ssc = ssc
    def run(self):
        ssc.start()
        ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [26]:
spark

In [27]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import IntegerType
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import PipelineModel

========= 2019-05-23 12:21:20 =========
+----------+--------------------+--------------+------------+--------------------+--------------+-----------+----------+
|   book_id|          book_title|     review_id|review_score|         review_text|  review_title|review_user| timestamp|
+----------+--------------------+--------------+------------+--------------------+--------------+-----------+----------+
|0312577230|The Great Alone: ...|R1X0O6Q8ZR4JHO|           4|One of the best b...|Excellent book|     KC Mac|1558606864|
+----------+--------------------+--------------+------------+--------------------+--------------+-----------+----------+

+----------+--------------------+--------------+------------+--------------------+--------------+-----------+----------+----+
|   book_id|          book_title|     review_id|review_score|         review_text|  review_title|review_user| timestamp|pred|
+----------+--------------------+--------------+------------+--------------------+--------------+-----

In [28]:
globals()['models_loaded'] = False
path = "lrm_bin.model"

def predict(df):
    # Replace this with something smarter
    
    #my_model = PipelineModel.load(path)
    #return model.predict(df)
    return 1
    #sameModel.predict(array([0.0, 1.0]))
    
    #if any([x in df.review_text for x in ['not', 'bad', 'terrible', 'zero', 'one']]):
    #    return 1
    #return 5

In [29]:
#model = PipelineModel.load(path)

In [30]:
#globals()['models_loaded'] = False

#def predict2(df):
    # Replace this with something smarter
#    if any([x in df.review_text for x in ['not', 'bad', 'terrible', 'zero', 'one']]):
#        return 1
#    return 5

predict_udf = udf(predict, IntegerType())

def process(time, rdd):
    if rdd.isEmpty():
        return
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
    
    # Utilize our predict function
    df_withpreds = df.withColumn("pred", predict_udf(struct([df[x] for x in df.columns])))
    df_withpreds.show()
    
    # Normally, you wouldn't use a Python function to predict
    # But an MLlib model you've built and saved with Spark
    
    # In this case, you need to prevent loading your model in every call to "process" as follows:
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = 'my_model' # Replace this with:    [...].load('my_logistic_regression')
        #globals()['my_model'] = PipelineModel.load(path) # Replace this with:    [...].load('my_logistic_regression')
        globals()['models_loaded'] = True
        
    # Predict using the model: 
    # df_result = globals()['my_model'].transform(df)
    # df_result.show()

In [31]:
ssc = StreamingContext(sc, 10)

In [32]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [33]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/miniconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipython-input-25-19fbf29765bb>", line 8, in run
    ssc.start()
  File "/Users/admin/Documents/spark/spark-2.4.0-bin-hadoop2.7/python/pyspark/streaming/context.py", line 182, in start
    self._jssc.start()
  File "/Users/admin/Documents/spark/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/admin/Documents/spark/spark-2.4.0-bin-hadoop2.7/python/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/admin/Documents/spark/spark-2.4.0-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py", line 328, in get_return_value
    format(target_id, ".", name), value)
py4j.protocol.Py4JJavaError: An error occurred while calling o982.start.
: java.lang.IllegalStateException:

In [10]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
